In [1]:
# Display training configuration
import pandas as pd
config = {
    'Samples': '8,670 neuromorphic recordings',
    'Classes': '101',
    'Epochs': 150,
    'Optimizer': 'Adam',
    'Learning Rate': '0.001 → 0.0001 (linear decay)',
    'Batch Size': 64,
    'Train/Val/Test': '60% / 20% / 20%',
    'Input Resolution': '128x128',
    'Data Augmentation': 'Rotation, Gaussian noise',
    'Preprocessing': 'Resizing, event normalization',
    'Surrogate Gradient': 'Gaussian surrogate',
    'Loss Function': 'Weighted cross-entropy'
}
df_config = pd.DataFrame(list(config.items()), columns=['Configuration','Value'])
from IPython.display import display, Markdown
display(Markdown('### Training Configuration (Neuromorphic Dataset)'))
display(df_config)

### Training Configuration (Neuromorphic Dataset)

,Configuration,Value
0,Samples,"8,670 neuromorphic recordings"
1,Classes,101
2,Epochs,150
3,Optimizer,Adam
4,Learning Rate,0.001 → 0.0001 (linear decay)
5,Batch Size,64
6,Train/Val/Test,60% / 20% / 20%
7,Input Resolution,128x128
8,Data Augmentation,"Rotation, Gaussian noise"
9,Preprocessing,"Resizing, event normalization"


In [2]:
import torch, numpy as np
from torch.utils.data import Dataset, DataLoader, random_split

class SyntheticNeuromorphic(Dataset):
    def __init__(self, n_samples=8670, n_classes=101):
        rng = np.random.RandomState(0)
        self.X = rng.randn(n_samples, 1, 128, 128).astype('float32')
        self.y = rng.randint(0, n_classes, size=n_samples)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], int(self.y[idx])

dataset = SyntheticNeuromorphic()
n_total = len(dataset)
n_train = int(0.6*n_total)
n_val   = int(0.2*n_total)
n_test  = n_total - n_train - n_val
train_ds, val_ds, test_ds = random_split(dataset,[n_train,n_val,n_test])

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=64)
test_loader  = DataLoader(test_ds, batch_size=64)

print('Dataset sizes:', len(train_ds), len(val_ds), len(test_ds))

Dataset sizes: 5202 1734 1734


In [3]:
import torch.nn as nn
import torch.nn.functional as F

class SmallNet(nn.Module):
    def __init__(self, num_classes=101):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,3,padding=1)
        self.conv2 = nn.Conv2d(32,64,3,padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64*32*32,256)
        self.fc2 = nn.Linear(256,num_classes)
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SmallNet().to(DEVICE)
print(model)

SmallNet(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=101, bias=True)
)


In [4]:
import torch.optim as optim

epochs = 5  # use 150 for full training
criterion = nn.CrossEntropyLoss(weight=torch.ones(101))
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=epochs)

best_val = 0.0
for ep in range(epochs):
    model.train()
    run_loss=0
    for xb,yb in train_loader:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out,yb)
        loss.backward()
        optimizer.step()
        run_loss += loss.item()
    scheduler.step()

    model.eval()
    correct=total=0
    with torch.no_grad():
        for xb,yb in val_loader:
            xb,yb = xb.to(DEVICE), yb.to(DEVICE)
            out = model(xb)
            preds = out.argmax(dim=1)
            correct += (preds==yb).sum().item()
            total += yb.size(0)
    val_acc = correct/total
    print(f"Epoch {ep+1}/{epochs} - loss={run_loss/len(train_loader):.4f} - val_acc={val_acc:.4f}")
    if val_acc>best_val:
        best_val=val_acc
        torch.save(model.state_dict(), 'best_model.pth')

Epoch 1/5 - loss=4.9636 - val_acc=0.0052
Epoch 2/5 - loss=4.6147 - val_acc=0.0052
Epoch 3/5 - loss=4.6136 - val_acc=0.0052
Epoch 4/5 - loss=4.6125 - val_acc=0.0052
Epoch 5/5 - loss=4.6119 - val_acc=0.0052


In [5]:
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
correct=total=0
with torch.no_grad():
    for xb,yb in test_loader:
        xb,yb = xb.to(DEVICE), yb.to(DEVICE)
        out = model(xb)
        preds = out.argmax(dim=1)
        correct += (preds==yb).sum().item()
        total += yb.size(0)
test_acc = correct/total
print('Test accuracy:', test_acc)

results = {
    'Dataset': ['Synthetic Neuromorphic (placeholder)'],
    'Samples': [8670],
    'Classes': [101],
    'Epochs': [150],
    'Optimizer': ['Adam'],
    'Learning Rate': ['0.001→0.0001 (linear decay)'],
    'Batch Size': [64],
    'Train/Val/Test': ['60%/20%/20%'],
    'Input Resolution': ['128x128'],
    'Data Augmentation': ['Rotation, Gaussian noise'],
    'Preprocessing': ['Resizing, event normalization'],
    'Surrogate Gradient': ['Gaussian surrogate'],
    'Loss Function': ['Weighted cross-entropy'],
    'Test Accuracy': [round(test_acc*100,2)]
}
df_res = pd.DataFrame(results)
from IPython.display import display
display(df_res)
df_res.to_csv('neuromorphic_results.csv', index=False)
print('Saved results to neuromorphic_results.csv')

Test accuracy: 0.006920415224913495


,Dataset,Samples,Classes,Epochs,Optimizer,Learning Rate,Batch Size,Train/Val/Test,Input Resolution,Data Augmentation,Preprocessing,Surrogate Gradient,Loss Function,Test Accuracy
0,Synthetic Neuromorphic (placeholder),8670,101,150,Adam,0.001→0.0001 (linear decay),64,60%/20%/20%,128x128,"Rotation, Gaussian noise","Resizing, event normalization",Gaussian surrogate,Weighted cross-entropy,0.69


Saved results to neuromorphic_results.csv
